# Data cleaning and reformating

In [268]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset\


import pandas as pd
import numpy as np
import os

from random import shuffle, seed

from sklearn import svm

### Hyperparameter
window_size: The number of timesteps in one window (e.g. how many rows in one window).

channel: The number of features in one window. Similar to image channels (RGB).

batch_size: The numebr of windows in one batch.

learning_rate: How fast the model learns.

In [269]:
window_size = 10
channel = 1
batch_size = 32
learning_rate = 0.001

In [270]:
index_dict = {'Carrying_5':0, 'Carrying_10':1, 'Carrying_20':2}

In [271]:
print(index_dict)

{'Carrying_5': 0, 'Carrying_10': 1, 'Carrying_20': 2}


In [272]:
header = ['TimeSec', 'Sensor', 'Quatx', 'Quaty', 'Quatz', 'Quat0', 'Heading',
       'Pitch', 'Roll', 'LinAccx', 'LinAccy', 'LinAccz', 'Vbat', 'Accx',
       'Accy', 'Accz', 'Gyrox', 'Gyroy', 'Gyroz']

---
### Go to 
<a href=#bookmark> Run all cell above</a>

In [273]:
directory = "./Spinetrack Data/data/"
file = []

for d in sorted(os.listdir(directory)):
    if d != '.DS_Store':
        files = directory + d + "/Processed_redone"
        name = []
        try:
            for f in sorted(os.listdir(files)):
                if f.startswith("Carrying_") and f.endswith(".csv"): 
                    name.append(files + "/" + f)
            file.append(name)
            print("%s number of activity: %s" %(d, len(name)))
        except:
            pass
# file

Alex_data number of activity: 3
Alexander_data number of activity: 3
Charlotte_data number of activity: 3
Christian_data number of activity: 0
Elias_data number of activity: 3
Jesse_data number of activity: 3
Jiyoo_data number of activity: 3


In [274]:
frame = []
for person in file:
    for file_path in person:
        activity_lst = file_path.split("/")[-1].split('.')[0]
        
        activity_name = activity_lst
        print(activity_name)
        print("processing: ", file_path)
        if activity_name in index_dict.keys():
            df = pd.read_csv(file_path, error_bad_lines=False)
            df.columns = header
            df["activity"] = [index_dict[activity_name] for i in range(len(df))]
            frame.append(df)
            
            # mkdir if not exist. Save to local csv file
#             if not (os.path.exists('./Spinetrack Data/Yibin_Processed/' + category + folder_name)):
#                 os.makedirs('./Spinetrack Data/Yibin_Processed/' + category + folder_name)
#             csv_name = '/' + activity_name + '.csv'
#             folder_name = file_path.split("/")[-3] # person's name 
#             category = 'data/' # data or task
#             file_name = './Spinetrack Data/Yibin_Processed/' + category + folder_name + csv_name
#             df.to_csv(file_name) # save csv processed file to local
            
            #print(file_path)
data_df = pd.concat(frame)

result_df = data_df

Carrying_10
processing:  ./Spinetrack Data/data/Alex_data/Processed_redone/Carrying_10.csv
Carrying_20
processing:  ./Spinetrack Data/data/Alex_data/Processed_redone/Carrying_20.csv
Carrying_5
processing:  ./Spinetrack Data/data/Alex_data/Processed_redone/Carrying_5.csv
Carrying_10
processing:  ./Spinetrack Data/data/Alexander_data/Processed_redone/Carrying_10.csv
Carrying_20
processing:  ./Spinetrack Data/data/Alexander_data/Processed_redone/Carrying_20.csv
Carrying_5
processing:  ./Spinetrack Data/data/Alexander_data/Processed_redone/Carrying_5.csv
Carrying_10
processing:  ./Spinetrack Data/data/Charlotte_data/Processed_redone/Carrying_10.csv
Carrying_20
processing:  ./Spinetrack Data/data/Charlotte_data/Processed_redone/Carrying_20.csv
Carrying_5
processing:  ./Spinetrack Data/data/Charlotte_data/Processed_redone/Carrying_5.csv
Carrying_10
processing:  ./Spinetrack Data/data/Elias_data/Processed_redone/Carrying_10.csv
Carrying_20
processing:  ./Spinetrack Data/data/Elias_data/Proces

---
# Data processing and deep learning model

In [275]:
# Process original dataset, create windows (window_size samples(rows), about 1 second)
data = []
window = 1
while window*window_size < len(result_df):
    data_window = result_df[(window - 1)*window_size:window*window_size]
    data.append(data_window.values)
    window += 1
#data

In [276]:
len(data)

8061

In [277]:
# delete window if multiple activities and sensors presents
cleaned_data = []
for i in data:
    previous_activity = -1
    previous_sensor = -1
    for j in i:
        current_activity = j[19]
        current_sensor = j[1]
        if (previous_activity != -1) and (current_activity != previous_activity):
            print("data contains different activities! Window droped")
            break
#         elif (previous_sensor != -1) and (current_sensor != previous_sensor):
#             print("data contains different sensors! Window droped")
#             break
        else:
            previous_activity = current_activity
            previous_sensor = current_sensor
    else:
        cleaned_data.append(i)

data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped
data contains different activities! Window droped


In [278]:
len(cleaned_data)

8044

In [279]:
cleaned_data[0].shape

(10, 20)

In [280]:
# shuffle the data
seed(101)
shuffle(cleaned_data)
#cleaned_data

In [281]:
# extract label from each window
labels = []
for i in cleaned_data:
    label = i[0][19]
    labels.append(label)
labels = np.array(labels)
#labels

In [282]:
# extract features from each window
features = []
for i in cleaned_data:
    new = np.delete(i, 19, 1)
    features.append(new)
features = np.array(features)
#features

In [283]:
features.shape

(8044, 10, 19)

In [284]:
# X_train = np.resize(features[:int(len(cleaned_data)*0.8)], (int(len(cleaned_data)*0.8), -1))
# Y_train = labels[:int(len(cleaned_data)*0.8)]
# X_test = np.resize(features[int(len(cleaned_data)*0.8):], ((features.shape[0] - int(len(cleaned_data)*0.8)), -1))
# Y_test = labels[int(len(cleaned_data)*0.8):]

# clf = svm.SVC()
# clf.fit(X_train, Y_train)

In [285]:
# from sklearn import metrics
# Y_predict = clf.predict(X_test)
# metrics.accuracy_score(Y_test, Y_predict)

In [286]:
# combine the features and labels
k = list(zip(features, labels))
activity_data = pd.DataFrame(k)
activity_data.columns = ['features', 'labels']
activity_data

,features,labels
0,"[[30.502, 3.0, -0.18, 0.03, 0.02, -0.98, 173.2...",0.0
1,"[[51.61, 6.0, 0.161, 0.03, 0.02, -0.99, 212.54...",2.0
2,"[[36.451, 3.0, -0.9940000000000001, 0.03, -0.1...",0.0
3,"[[4.9510000000000005, 7.0, 0.431, 0.19, -0.14,...",0.0
4,"[[42.03, 3.0, -0.7509999999999999, 0.06, -0.05...",1.0
5,"[[14.633, 3.0, 0.289, -0.1, -0.08, -0.95, 227....",1.0
6,"[[50.248999999999995, 2.0, 0.513, -0.11, -0.04...",2.0
7,"[[25.961, 1.0, 0.5710000000000001, 0.03, -0.05...",2.0
8,"[[10.77, 6.0, 0.873, 0.08, 0.1, -0.47, 317.47,...",2.0
9,"[[11.544, 3.0, -0.7390000000000001, -0.07, 0.0...",2.0


In [287]:
activity_data['features'][8043].shape

(10, 19)

In [288]:
# check the size of activity. The final output of neural net 
# has to have max_index + 1 output
max_index = activity_data['labels'].max()
label_size = int(max_index + 1)
# label_size

---
## Load data

In [289]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()
if train_on_gpu:
    print("CUDA is available! Training on GPU.")
else:
    print("CUDA is not available. Training on CPU...")
# torch.cuda.current_device()

CUDA is available! Training on GPU.


In [290]:
# split the data to test, validation, and train
valid_size = 0.1
test_size = 0.1
activity_data.columns = ["features", "labels"]
activity_data_train = activity_data[:int(len(activity_data)*(1-valid_size-test_size))]
activity_data_valid = activity_data[int(len(activity_data)*(1-valid_size-test_size)):int(len(activity_data)*(1-test_size))]
activity_data_test = activity_data[int(len(activity_data)*(1-test_size)):]
# activity_data_train.to_csv("./activity_data_train.csv", encoding='utf-8-sig')
# activity_data_valid.to_csv("./activity_data_valid.csv", encoding='utf-8-sig')
# activity_data_train.to_csv("./activity_data_test.csv", encoding='utf-8-sig')

In [291]:
activity_data_train.shape

(6435, 2)

In [292]:
# define our dataset in pytorch
class DatasetSpineTrack(Dataset):
    
    def __init__(self, file, transform=None):
        #self.data = pd.read_csv(file_path)
        self.data = file
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        # load image as ndarray type (Height * Width * Channels)
        # be carefull for converting dtype to np.uint8 [Unsigned integer (0 to 255)]
        # in this example, i don't use ToTensor() method of torchvision.transforms
        # so you can convert numpy ndarray shape to tensor in PyTorch (H, W, C) --> (C, H, W)
        
        features = torch.tensor(self.data["features"].iloc[index])
        features = features.view(channel, window_size, -1) 
        labels = torch.tensor(self.data["labels"].iloc[index], dtype=torch.long)
        #print(labels.type())
        
#         if self.transform is not None:
#             image = self.transform(image)
            
        return features, labels

In [293]:
# construct training and testing dataset in csv
# train_dataset = DatasetSpineTrack("./activity_data_train.csv")
# valid_dataset = DatasetSpineTrack("./activity_data_valid.csv")
# test_dataset = DatasetSpineTrack("./activity_data_test.csv")
train_dataset = DatasetSpineTrack(activity_data_train)
valid_dataset = DatasetSpineTrack(activity_data_valid)
test_dataset = DatasetSpineTrack(activity_data_test)
feature, label = train_dataset.__getitem__(0)
#feature
#label

In [294]:
# load data
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
validloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
testloader = DataLoader(test_dataset, batch_size=batch_size, drop_last=True)

---
## Network Architechture

In [259]:
# train_on_gpu = False
# train_on_gpu = True

In [260]:
class TimeCNN(nn.Module):
    def __init__(self, channel, label_size):
        super(TimeCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(channel, 32, kernel_size=(3, 3), stride=(2, 2), padding=(3, 3), bias=False)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        
        # bn1 = BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(128, 64, bias=True)
        self.fc2 = nn.Linear(64, label_size, bias=True)
        
        self.dropout = nn.Dropout(0.25)
        
    def forward(self, x):
        # add sequence of convolutional and max pooling layers
#         print("forward shape 1: ", x.shape)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # flatten image input
        x = x.view(batch_size, -1)
#         print("forward shape 2: ", x.shape)
        # add dropout layer
        x = self.dropout(x)
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # add 2nd hidden layer, with relu activation function
        x = self.fc2(x)
        return x
model = TimeCNN(channel, label_size)
model = model.double()
if train_on_gpu:
    model.cuda()
print(model)

TimeCNN(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(3, 3), bias=False)
  (conv2): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (conv3): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=128, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=3, bias=True)
  (dropout): Dropout(p=0.25)
)


In [298]:
model = models.resnet18(pretrained=False)
# window_size channels
# model.conv1 = torch.nn.Conv2d(window_size, batch_size, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.conv1 = torch.nn.Conv2d(channel, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = torch.nn.Linear(512, label_size, bias=True)
model.add_module("dropout", torch.nn.Dropout(p=0.5))
model = model.double()

# move tensors to GPU is CUDA is available
if train_on_gpu:
    model.cuda()
print(model)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

---
### Run all above 
<a name='bookmark' />

---
## Training the Network

Remember to look at how the training and validation loss decreases over time; if the validation loss ever increases it indicates possible overfitting.

In [297]:
import time
start_time = time.time()

criterion = nn.CrossEntropyLoss()
#criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# number of epochs to train the model
n_epochs = 10

valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):
    
    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train() 
    for features, labels in trainloader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            features, labels = features.cuda(), labels.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(features)
        # calculate the batch loss
        loss = criterion(output, labels)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*features.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval()
    for features, labels in validloader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            features, labels = features.cuda(), labels.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(features)
        # calculate the batch loss
        loss = criterion(output, labels)
        # update average validation loss 
        valid_loss += loss.item()*features.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(trainloader.sampler)
    valid_loss = valid_loss/len(validloader.sampler)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_Spinetrack_data_carrying.pt')
        valid_loss_min = valid_loss

# output running time
running_time = time.time() - start_time
sec = running_time % 60
miniute = running_time / 60
print("Training time: {} min {} sec".format(int(miniute), int(sec)))

Epoch: 1 	Training Loss: 0.063313 	Validation Loss: 0.453311
Validation loss decreased (inf --> 0.453311).  Saving model ...
Epoch: 2 	Training Loss: 0.036335 	Validation Loss: 0.648362
Epoch: 3 	Training Loss: 0.044681 	Validation Loss: 0.522657
Epoch: 4 	Training Loss: 0.030796 	Validation Loss: 0.536758
Epoch: 5 	Training Loss: 0.049618 	Validation Loss: 0.572263
Epoch: 6 	Training Loss: 0.042538 	Validation Loss: 0.499991
Epoch: 7 	Training Loss: 0.016337 	Validation Loss: 0.682830
Epoch: 8 	Training Loss: 0.052441 	Validation Loss: 0.396683
Validation loss decreased (0.453311 --> 0.396683).  Saving model ...
Epoch: 9 	Training Loss: 0.032995 	Validation Loss: 0.530069
Epoch: 10 	Training Loss: 0.031664 	Validation Loss: 0.622748
Training time: 20 min 14 sec


In [265]:
train_on_gpu = False

In [266]:
class TimeCNN(nn.Module):
    def __init__(self, channel, label_size):
        super(TimeCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(channel, 32, kernel_size=(3, 3), stride=(2, 2), padding=(3, 3), bias=False)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        
        # bn1 = BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(128, 64, bias=True)
        self.fc2 = nn.Linear(64, label_size, bias=True)
        
        self.dropout = nn.Dropout(0.25)
        
    def forward(self, x):
        # add sequence of convolutional and max pooling layers
#         print("forward shape 1: ", x.shape)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # flatten image input
        x = x.view(batch_size, -1)
#         print("forward shape 2: ", x.shape)
        # add dropout layer
        x = self.dropout(x)
        # add 1st hidden layer, with relu activation function
        x = F.relu(self.fc1(x))
        # add dropout layer
        x = self.dropout(x)
        # add 2nd hidden layer, with relu activation function
        x = self.fc2(x)
        return x
model = TimeCNN(channel, label_size)
model = model.double()
if train_on_gpu:
    model.cuda()

###  Load the Model with the Lowest Validation Loss

In [299]:
# load on gpu
# model.load_state_dict(torch.load('model_Spinetrack_3.pt'))

# load on cpu
model.load_state_dict(torch.load('model_Spinetrack_data_carrying.pt', map_location=lambda storage, loc: storage))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

---
## Test the Trained Network

Test your trained model on previously unseen data! A "good" result will be a result that gets more than 70% accuracy on these test data.

In [300]:
# Release all the GPU memory cache that can be freed
torch.cuda.empty_cache() 

### Overall accuracy and each class accuracy

In [301]:
# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(len(index_dict)))
class_total = list(0. for i in range(len(index_dict)))

criterion = nn.CrossEntropyLoss()
#criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

model.eval()
torch.no_grad()
# iterate over test data
for features, labels in testloader:
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        features, labels = features.cuda(), labels.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(features)
    # calculate the batch loss
    loss = criterion(output, labels)
    # update test loss 
    test_loss += loss.item()*features.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(batch_size):
        try:
            label = labels.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1
        except:
            pass

# average test loss
test_loss = test_loss/len(testloader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(len(index_dict)):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            list(index_dict.keys())[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (list(index_dict.keys())[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.659969

Test Accuracy of Carrying_5: 90% (238/262)
Test Accuracy of Carrying_10: 83% (244/293)
Test Accuracy of Carrying_20: 86% (212/245)

Test Accuracy (Overall): 86% (694/800)


### Overall accuracy (different calculation method)

In [ ]:
# Turn off gradients for validation, saves memory and computations
torch.no_grad()
accuracy = 0
for features, labels in testloader:
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        features, labels = features.cuda(), labels.cuda()
    loss = model(features)
    test_loss += criterion(loss, labels)

#     ps = torch.exp(loss)
    top_p, top_class = loss.topk(1, dim=1)
    equals = top_class == labels.view(*top_class.shape)
    accuracy += torch.mean(equals.type(torch.FloatTensor))
print("Test Accuracy: {:.3f}".format(accuracy/len(testloader)))

In [ ]:
# TODO: Figure out pulling_OneH